In [154]:
import copy
#Wumpus puzzle class that holds important information about the puzzle
class WumpusPuzzle:
    def __init__(self, s, a, q, qa, p):
        self.puzzleSize = s
        self.arrows = a
        self.query = q
        self.queryAnswer = qa
        self.path = p

In [155]:
#Clause class which holds the disjuncted list of literals
class Clause:
    def __init__(self, l):
        # list of Literal objects
        self.literals = list(l)
        #duplicate literals are removed from clauses when created
        self.removeDuplicates()

    #prints out a string representation of the clause for debugging
    def __str__(self):
        finalStr = ""
        for i in range(len(self.literals)):
            finalStr += str(i) +". " + str(self.literals[i].negation) + " function " + str(self.literals[i].predicateFunc) + ", variables: "
            for j in range(len(self.literals[i].terms)):
                finalStr += str(self.literals[i].terms[j]) + " "
            finalStr += "\n"
        return finalStr


    def removeDuplicates(self):
        def lit_key(l):
            return (l.predicateFunc, l.negation, tuple(str(t) for t in l.terms))
        #Removing duplicates from the list of literals
        seen = set()
        unique = []
        for lit in self.literals:
            if lit not in seen:
                seen.add(lit)
                unique.append(lit)
        self.literals = unique

        #Sorting literals so hashing works properly
        self.literals.sort(key=lit_key)

    #custom equality function for comparing clauses in sets
    def __eq__(self, other):
        if other is None:
            return False
        # clauses are equal if they have the same set of literals
        return len(self.literals) == len(other.literals) and all(a == b for a, b in zip(self.literals, other.literals))

    #custom hashing function for storing clauses in sets
    def __hash__(self):
        return hash(frozenset(self.literals))



In [156]:
#Literal class which holds a predicate and terms
class Literal:
    def __init__(self, n, f, t):
        #boolean
        self.negation = n
        #predicate function
        self.predicateFunc = f
        #list of terms
        self.terms = t


    #Custom equality function for use in unification
    def __eq__(self, otherLiteral):
        if(self.negation == otherLiteral.negation and self.predicateFunc == otherLiteral.predicateFunc and termsEqual(self, otherLiteral)):
            return True
        return False

    #custom hashing for storing literals
    def __hash__(self):
        return hash((self.negation, self.predicateFunc, tuple(self.terms)))

    #Given a theta by the unification algorithm, replaces all unified variables
    def replaceVariables(self, dict):
        for replacementVariable in dict.keys():
            for predVariable in range(len(self.terms)):
                if(self.terms[predVariable].value == replacementVariable):
                    print(self.terms[predVariable].value + " being replaced by " + dict[replacementVariable])
                    self.terms[predVariable].value = dict[replacementVariable]


#Checks if two literals are the negation of one another
def literalsCancel(literal1, literal2):
    if(literal1.negation != literal2.negation and literal1.predicateFunc == literal2.predicateFunc):
        return True
    else:
        return False

#Checks if every term in literal1 is equal to every term in literal2
def termsEqual(literal1, literal2):
    for i in range(len(literal1.terms)):
            if literal1.terms[i] != literal2.terms[i]:
                return False
    return True

In [157]:
#Term class which holds information about the term and its value
class Term:
    def __init__(self, v, c, s, value):
        self.isVariable = v
        self.isConstant = c
        self.isSkolem = s
        self.value = value

    def __str__(self):
        return str(self.value)

    def __eq__(self, other):
        if other is None:
            return False
        return self.value == other.value

    def __hash__(self):
        return hash(self.value)

In [158]:
#Skolem function which holds another term and a function (e,g, F())
class SkolemFunction:
    def __init__(self, term, function):
        self.term = term
        self.function = function

In [159]:
#Resolution algorithm
def resolution(KB, query):
    clauses = copy.deepcopy(KB)
    clauses.add(query)
    newLength = 0
    while(True):
        new = set()
        i = 0
        resolved = 0
        prevLength = newLength
        #iterating through all pairs of clauses
        for clause in clauses:
            for clause2 in clauses:
                if(clause != clause2):
                    i+=1
                    #attempt to resolve two clauses
                    resolvent = resolve(clause, clause2)
                    #Makes sure a clause was actually resolved
                    if(resolvent != None):
                        #Empty set resolved, meaning the resolution algorithm returns True
                        if(len(resolvent.literals) == 0):
                            return True
                        else:
                            #Resolved clause added to set
                            new.add(resolvent)
                            resolved += 1
        #Makes sure all clauses being added are actually new
        newResolvents = new - clauses
        #If no new clauses were resolved, resolution returns False
        if not newResolvents:
            return False
        #unions the set of clauses with the new resolvents
        clauses |= newResolvents
        newLength = len(clauses)
        print("PrevLength: " , prevLength, "NewLength:", newLength, "resolved:", resolved, "unresolved:", i-resolved )



#resolving two clauses
def resolve(c1, c2):
    literalRemoved = False
    newLiterals = []
    newLiterals += c1.literals + c2.literals
    #iterates through each literal pair in the clauses
    for literal1 in c1.literals:
        for literal2 in c2.literals:
            #If the literals are negations of the same predicate, try to unify them
            if literalsCancel(literal1, literal2):
                theta = {}
                theta = unify(literal1, literal2, theta)
                #unification worked, proceed with removing literals
                if(theta != "fail"):
                    literalRemoved = True
                    # for literalSub1 in clause1.literals:
                    #     for literalSub2 in clause2.literals:
                    #         literalSub1.replaceVariables(theta)
                    #         literalSub2.replaceVariables(theta)

                    newLiterals.remove(literal1)
                    newLiterals.remove(literal2)
    #if no literal was removed, don't return anything (clauses didn't resolve)
    if(not literalRemoved):
        return None
    else:
        newClause = Clause(newLiterals)
        return newClause





In [160]:
# Perform substitution to make x and y the same
def unify(x, y, theta):
    if theta == "fail":
        return "fail"

    if (x == y):
        return theta

    elif (isinstance(x,Term) and x.isVariable):
        return unifyVar(x, y, theta)
    elif (isinstance(y,Term) and y.isVariable):
        return unifyVar(y, x, theta)


    elif isinstance(x, Literal) and isinstance(y, Literal):
        if(x.predicateFunc == y.predicateFunc):
            return unify(x.terms, y.terms, theta)
        else:
            return "fail"


    elif isinstance(x, list) and isinstance(y, list):
        if len(x) != len(y):
            return "fail"
        for i, j in zip(x, y):
            theta = unify(i, j, theta)
            if theta == "fail":
                return "fail"
        return theta

    else:
        return "fail"


def unifyVar(var, x, theta):
    if var in theta:
        return unify(theta[var],x,theta)
    elif x in theta:
        return unify(var,theta[x],theta)
    #Maybe need to have the occur-check call here and write occur-check function? Says not always done in the textbook
    else:
        new = theta.copy()
        new[var.value] = x.value
        return new


In [161]:
PuzzleFile = 'Caves/_wumpus_paths_/easy/path_e2.txt'

def processFile(fileName):
    with open(fileName, 'r', encoding='utf-8-sig') as file:
        #processes the file to store the number of arrows and the size of the grid
        content = file.read().strip()
        result = content.split("\n")
        result2 = result[0].split("GRID: ")
        xAxis = int(result2[1].split("x")[0])
        yAxis = int(result2[1].split("x")[1])
        size = [xAxis, yAxis]
        result3 = result[1].split("ARROWS: ")
        arrows = int(result3[1])
        index = 3
        path = result[index]
        pathArr = []
        #processes the path, storing the value of each cell and if breeze or stench was detected
        while(path != ''):
            pathStr = path.strip().split("(")[1].split(")")
            xVal = int(pathStr[0].split(",")[0])
            yVal = int(pathStr[0].split(",")[1])
            breezeVal = pathStr[1].split("B:")[1][0]
            stenchVal = pathStr[1].split("S:")[1][0]
            pathVal = [[xVal, yVal], breezeVal, stenchVal]
            pathArr.append(pathVal)
            index+=1
            path = result[index]
        #stores the queried cell
        queryResult = result[index+1].split("QUERY: ")[1].split(',')
        query = [int(queryResult[0][1:]), int(queryResult[1][:-1])]
        resolution = result[index+2].split("RESOLUTION:")[1].strip()
        puzzle = WumpusPuzzle(size, arrows, query, resolution, pathArr)
        return puzzle

puzzle = processFile(PuzzleFile)

[25, 25]
2
[[[0, 0], 'F', 'F'], [[1, 0], 'F', 'F'], [[2, 0], 'F', 'T'], [[2, 1], 'F', 'F'], [[2, 2], 'T', 'F'], [[1, 2], 'F', 'F'], [[1, 3], 'F', 'F'], [[2, 3], 'F', 'T']]
[2, 4]
RISKY


In [162]:
#Rules being setup for the wumpus world
#1. All locations visited don't have a wumpus or pit in them
#2. All locations visited with no b/s have no wumpuses or pits in the cardinal directions
#3. ALl locations visited with a b/s have a wumpus or pit in the cardinal direction
KB = set()
xMax = puzzle.puzzleSize[0]
yMax = puzzle.puzzleSize[1]
for i in range(len(puzzle.path)):
    c = puzzle.path[i][0]
    cell = str(c[0]) + "," + str(c[1])
    breeze = puzzle.path[i][1]
    stench = puzzle.path[i][2]
    #Setting up rule #1
    t = Term(False, True, False, cell)
    literal = Literal(False, "Wumpus", [t])
    clause = Clause([literal])
    KB.add(clause)
    literal = Literal(False, "Pit", [t])
    clause = Clause([literal])
    KB.add(clause)
    #Rule 2
    if(breeze == "F"):
        literalList = []
        #Checks cells in cardinal directions, making false pit literals for each one
        for x in [c[0]-1, c[0]+1]:
            if(x>=0 and x < xMax):
                n = [x,c[1]]
                newCell = str(n[0]) + "," + str(n[1])
                t = Term(False, True, False, newCell)
                literal = Literal(False, "Pit", [t])
                literalList.append(literal)
        for y in [c[1]-1, c[1]+1]:
            if(y>=0 and y < yMax):
                n = [c[0],y]
                newCell = str(n[0]) + "," + str(n[1])
                t = Term(False, True, False, newCell)
                literal = Literal(False, "Pit", [t])
                literalList.append(literal)
        #all pit literals are put into a clause and added to the knowledge base
        clause = Clause(literalList)
        KB.add(clause)
    #Rule 3
    else:
        literalList = []
        for x in [c[0]-1, c[0]+1]:
            if(x>=0 and x < xMax):
                n = [x,c[1]]
                newCell = str(n[0]) + "," + str(n[1])
                t = Term(False, True, False, newCell)
                literal = Literal(True, "Pit", [t])
                literalList.append(literal)
        for y in [c[1]-1, c[1]+1]:
            if(y>=0 and y < yMax):
                n = [c[0],y]
                newCell = str(n[0]) + "," + str(n[1])
                t = Term(False, True, False, newCell)
                literal = Literal(True, "Pit", [t])
                literalList.append(literal)
        clause = Clause(literalList)
        KB.add(clause)
    #Rule 2
    if(stench == "F"):
        literalList = []
        for x in [c[0]-1, c[0]+1]:
            if(x>=0 and x < xMax):
                n = [x,c[1]]
                newCell = str(n[0]) + "," + str(n[1])
                t = Term(False, True, False, newCell)
                literal = Literal(False, "Wumpus", [t])
                literalList.append(literal)
        for y in [c[1]-1, c[1]+1]:
            if(y>=0 and y < yMax):
                n = [c[0],y]
                newCell = str(n[0]) + "," + str(n[1])
                t = Term(False, True, False, newCell)
                literal = Literal(False, "Wumpus", [t])
                literalList.append(literal)
        clause = Clause(literalList)
        KB.add(clause)
    #Rule 3
    else:
        literalList = []
        for x in [c[0]-1, c[0]+1]:
            if(x>=0 and x < xMax):
                n = [x,c[1]]
                newCell = str(n[0]) + "," + str(n[1])
                t = Term(False, True, False, newCell)
                literal = Literal(True, "Wumpus", [t])
                literalList.append(literal)
        for y in [c[1]-1, c[1]+1]:
            if(y>=0 and y < yMax):
                n = [c[0],y]
                newCell = str(n[0]) + "," + str(n[1])
                t = Term(False, True, False, newCell)
                literal = Literal(True, "Wumpus", [t])
                literalList.append(literal)
        clause = Clause(literalList)
        KB.add(clause)

#Creates a term for the queried cell
q = puzzle.query
queryStr = str(q[0]) + "," + str(q[1])
q = Term(False, True, False, queryStr)



[[2, 3], 'F', 'T']
[2, 4]


In [163]:
#answers the query using resolution given the puzzle knowledge base and the cell term
def answerQuery(KB, cellTerm):
    #Check for unsafeness
    negated = Literal(False, "Wumpus", [cellTerm])
    negatedClause = Clause([negated])
    noWumpusContradiction = resolution(KB, negatedClause)
    if(noWumpusContradiction == True):
        return "UNSAFE"
    negated = Literal(False, "Pit", [cellTerm])
    negatedClause = Clause([negated])
    noPitContradiction = resolution(KB, negatedClause)
    if(noPitContradiction == True):
        return "UNSAFE"

    #Check for safeness
    negated1 = Literal(True, "Wumpus", [cellTerm])
    negated2 = Literal(True, "Pit", [cellTerm])
    negatedClause = Clause([negated1, negated2])
    safteyContradiction = resolution(KB, negatedClause)
    if(safteyContradiction == True):
        return "SAFE"

    #unable to determine if safe or unsafe, returns RISKY
    return "RISKY"

print(answerQuery(KB, q))

PrevLength:  0 NewLength: 47 resolved: 28 unresolved: 1028
PrevLength:  47 NewLength: 69 resolved: 106 unresolved: 2056
PrevLength:  69 NewLength: 89 resolved: 210 unresolved: 4482
PrevLength:  89 NewLength: 99 resolved: 384 unresolved: 7448
PrevLength:  0 NewLength: 46 resolved: 26 unresolved: 1030
PrevLength:  46 NewLength: 64 resolved: 88 unresolved: 1982
PrevLength:  64 NewLength: 79 resolved: 154 unresolved: 3878
PrevLength:  79 NewLength: 85 resolved: 284 unresolved: 5878
PrevLength:  0 NewLength: 47 resolved: 28 unresolved: 1028
PrevLength:  47 NewLength: 65 resolved: 90 unresolved: 2072
PrevLength:  65 NewLength: 80 resolved: 156 unresolved: 4004
PrevLength:  80 NewLength: 86 resolved: 286 unresolved: 6034
RISKY


In [164]:


# KB = set()
#
# Tuna = Term(False, True, False, "Tuna")
# Jack = Term(False, True, False, "Jack")
# Curiosity = Term(False, True, False, "Curiosity")
# x = Term(True, False, False, "x")
# f = SkolemFunction(x, "F")
# fx = Term(False,False,True, f)
# g = SkolemFunction(x, "G")
# gx = Term(False,False,True, g)
# y = Term(True, False, False, "y")
# z = Term(True, False, False, "z")
#
#
#
#
# l = Literal(True, "Cat", [Tuna])
# rule6 = Clause([l])
#
# l = Literal(False, "Cat", [x])
# l2 = Literal(True, "Animal", [x])
# #test unification on different functions
# rule7 = Clause([l,l2])
#
#
# l = Literal(True,"Animal", [fx])
# l2 = Literal(True, "Loves", [gx, x])
# rule1 = Clause([l,l2])
#
#
#
# l = Literal(False, "Loves", [x,fx])
# l2 = Literal(True, "Loves", [gx, x])
# #test unification on skolems
# rule2 = Clause([l,l2])
#
#
# l = Literal(False, "Loves",[y,x])
# l2 = Literal(False, "Animal", [z])
# l3 = Literal(False, "Kills", [x,z])
# rule3 = Clause([l,l2,l3])
#
#
# l = Literal(False, "Animal", [x])
# l2 = Literal(True, "Loves", [Jack, x])
# rule4 = Clause([l,l2])
#
# l = Literal(True, "Kills", [Jack, Tuna])
# l2 = Literal(True, "Kills", [Curiosity, Tuna])
# #test unification on constants
# rule5 = Clause([l,l2])
#
# KB.add(rule1)
# KB.add(rule2)
# KB.add(rule3)
# KB.add(rule4)
# KB.add(rule5)
# KB.add(rule6)
# KB.add(rule7)
#
#
# l = Literal(False, "Kills", [Curiosity, Tuna])
# query = Clause([l])
#
# #Testing literal cancel checking function
# # p = Predicate([Jack,Tuna], Kills)
# # l = Literal(p, True)
# # p = Predicate([Jack,Tuna], Kills)
# # l2 = Literal(p, False)
# # print(literalsCancel(l,l2))
#
# #Testing unification
# # Variables
# theta = {}
# x2 = copy.deepcopy(x)
# theta = unify(x, x2, theta)
# print(theta)
#
# # Literals
# l = Literal(True, "Kills", [Curiosity, Tuna])
# l2 = Literal(True, "Kills", [x, y])
# theta = {}
# theta = unify(l, l2, theta)
# print(theta)
# # --- Test ---
# # theta = {}
# # print("Unify Literals:", unify(l1, l2, theta))
# # print("Unify Predicates:", unify(p1, p2, theta))
# # print("Unify Clauses:")
# # for l in rule7.literals:
# #     for l2 in rule6.literals:
# #         print(unify(l, l2, theta))
# #
# #
# # #
# newClauses = resolve(rule6, rule7)
# # # print(rule6)
# # # print(rule7)
# print(newClauses)
# print(rule3)
# newerClauses = resolve(newClauses, rule3)
# print(newerClauses)
# newerClauses2 = resolve(rule5, query)
# print(newerClauses2)
# #tester = resolution(KB, query)
# final = resolve(newerClauses, newerClauses2)
# print(final)